# Fourier Analysis Lab
All  of  the  programming  assignments  are  to  be  done  in  Python  using  additional  libraries  specified  in  the  assignments.  There  are many  libraries  available,  some  of  which  we  will  be  using,  and  you  are  welcome  to  use  them  with  one  exception:  if  the  library  or  a  function  within  it  performs  the  specific  function  you  are  asked  to  code,  you  may  not  use  that  other  than  perhaps  as  a  reference  to  compare  against. All  of  the  code  you  submit  must  be  your  own. You are welcome to turn in a completed jupyter notebook.

## The Basics

We have discussed the Fourier Transform in great detail in class. In this lab, we will implement many of the practical applications of the Fourier Transform. In order to do this, we need to know how the Fourier Transform is implemented in the Numpy library.

The following code can be used to perform the Fourier Transform and Inverse Fourier Transform.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#Initialize Data
data = np.zeros(300)
data[125:175] = 1
plt.plot(data);plt.title("Spatial Function");plt.show()

#Peform Fourier Transform
fourier = np.fft.fft(data)
#Plot the complex magnitude of the fourier transform
plt.plot(np.absolute(fourier));plt.title("Frequency Function");plt.show()

#Perform Inverse Fourier Transform
original = np.fft.ifft(fourier)
original = np.absolute(original)
plt.plot(original);plt.title("Back to Spatial");plt.show()

This is a very simple way to access and manipulate the fourier transform. Note that the frequency function defaults to the non centralized version for calculation. For visualization, you may choose to plot a centralized frequency function.

In [ ]:
#Centralize the plot
N = len(fourier)
xvals = range(-N/2+1,N/2+1)

#Take the second half of the data, reverse and append it to the front
result = np.fft.fftshift(fourier)

plt.plot(xvals,np.absolute(result));plt.title("Centralized Fourier");plt.show()

You can also easily perform the fourier transform on a two dimensional set of data as well. This will be very helpful for doing transforms on our images.

In [ ]:
#Initialize Data
data2 = np.zeros((300,300))
data2[140:160,130:170] = 1
plt.imshow(data2,cmap="gray",interpolation="none");plt.title("Spatial Function");plt.show()

#Peform Fourier Transform
fourier2 = np.fft.fft2(data2)
#Plot the complex magnitude of the fourier transform
plt.imshow(np.absolute(fourier2),cmap="gray",interpolation="none");plt.title("Frequency Function");plt.show()

#Perform Inverse Fourier Transform
original2 = np.fft.ifft2(fourier2)
original2 = np.absolute(original2)
plt.imshow(original2,cmap="gray",interpolation="none");plt.title("Back to Spatial");plt.show()

Again, you may also choose to centralize the fourier transform in 2D.

In [ ]:
#Centralize the plot
M,N = np.shape(fourier2)
xvals = range(-N/2+1,N/2+1)
yvals = range(-M/2+1,M/2+1)

result2 = np.fft.fftshift(fourier2)

plt.imshow(np.absolute(result2),cmap="gray",interpolation="none",extent=[xvals[0],xvals[-1],yvals[0],yvals[-1]])
plt.title("Centralized Fourier");plt.show()

Now that you have the basics down, you can start manipulating the data in useful ways.

To get full credit for this lab. Complete each of the tasks below.

## Task 1: 1-D Filtering

Design a 1-D low-pass filter to smooth the data in Signal.pkl. To help you with the basic structure of filtering programs, here is an
[example script](https://www.dropbox.com/s/1jsjyuc9e6k8r3i/freq_filter_script.py?dl=0) for performing the filtering.

In [ ]:
import matplotlib.pyplot as plt

import pickle
data = pickle.load(open('Signal.pkl','r'))

plt.plot(data);plt.title("Original Data");plt.show()

In [ ]:
def plotf(f,title=""):
    plt.plot(f)
    plt.title(title)
    plt.show()

def plotc(f,title=""):
    #Centralize the plot
    N = len(f)
    xvals = range(-N/2+1,N/2+1)

    #Take the second half of the data, reverse and append it to the front
    result = np.fft.fftshift(f)

    plt.plot(xvals,np.absolute(result));plt.title(title);plt.show()
    
# 1 Dimensional Filter Code
def transform1D(f,H):
    # Put f in frequency domaain
    F = np.fft.fft(f)
    
    # Perform transform
    G = F*H
    
    # Bring back into spacial domain
    g = np.fft.ifft(G)
    
    magnitude = np.absolute(g)
    return magnitude

def transformAudio(f, H):
    # Put f in frequency domaain
    F = np.fft.fft(f)
    
    # Perform transform
    G = F*H
    
    # Bring back into spacial domain
    g = np.fft.ifft(G)
    
    return g

def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

# Generate Gaussian Filter
N = len(data)
x = np.linspace(0,1, N)
sd = .01

H = gaussian(x,0,sd)

#Add mirroed negative values
vals = H[:N/2]
mirrored = vals[::-1]
H[(-N/2):] = mirrored

plotc(H, "Low Pass Filter")

g = transform1D(data,H)

plotf(g, "New Output")

## Task 2: Filtering Sound

You are given a noisy sound file called **NoisyAudio.wav**. Using a filter similar to the one you made in Task 1, remove as much noise as possible from the audio clip.

(Reminder: If sounddevice doesn't import, simply call *pip install sounddevice*)

In [ ]:
import time
from scipy.io import wavfile
import sounddevice as player

#Import data and convert to Mono channel audio
fs, data = wavfile.read("NoisyAudio.wav")
data = data[:,0]

# Generate Gaussian Filter
N = len(data)
x = np.linspace(0,1, N)
sd = .2

H = gaussian(x,0,sd)

#Add mirroed negative values
vals = H[:N/2]
mirrored = vals[::-1]
H[(-N/2):] = mirrored

plotc(H, "Filter")

plotf(data)

data = np.real(transformAudio(data,H))

plotf(data)

print "Done"

#Play the new audio
player.play(data, fs)

## Task 3: 2D Blurring and Sharpening

In previous labs, we have performed blurring and sharpening of 2D images. We did this by using specific kernels with this image. These processes can also be done more continuously using a filters in the frequency domain.

For a 2D image, a blurring effect can be implemented by applying a low-pass filter to the image in the frequency domain. In similar fashion, a sharpening effect can be implemented by applying a high-boost filter to the image in the frequency domain.

**Implement a blurring and sharpening effect using the frequency domain on a grayscale image of your choice.**

Again, to help you with the basic structure of filtering programs, reference the
[example script](https://www.dropbox.com/s/1jsjyuc9e6k8r3i/freq_filter_script.py?dl=0) for performing the filtering.

In [ ]:
def transform2D(f,H):

    # Convert image to a gray floating-point (double) image in range 0.0 to 1.0
    if f.ndim == 3:
        f = np.double(rgb2gray(f) )/255.0
    else:
        f = np.double(f)/255.0
 
    # Put f into frequency domain
    F = np.fft.fft2(f)
    
    # Perform Transform Function
    G = F*H
    
    # Bring Back into spatial domain
    g = np.fft.ifft2(G)
    
    magnitude = np.absolute(g)
    
    return magnitude

In [ ]:
from scipy.ndimage import imread

filename = "example.jpg"
im = imread(filename)

plt.imshow(im, cmap='gray', interpolation='none')
plt.show()

rows,cols = np.shape(im)

#Generate a Sharpening Kernel
X,Y=np.meshgrid(np.linspace(-10,10,cols),np.linspace(-10,10,rows));
mu,sigma=0,3;
G=np.exp(-((X-mu)**2+(Y-mu)**2)/(2.0*sigma**2))
S=G+1

sharpened = transform2D(im,S)
plt.imshow(sharpened, cmap='gray', interpolation='none')
plt.title("Sharpened");plt.show()

#Generate a blurring kernel
B = np.fft.fftshift(G)
blurred = transform2D(im,B)
plt.imshow(blurred, cmap='gray', interpolation='none')
plt.title("Blurred");plt.show()


## Task 4: Interference Pattern

The image **[interfere.png](https://faculty.cs.byu.edu/~farrell/cs450/interfere.png)** has an interference pattern of unknown spatial frequency, orientation and magnitude (it is, however, a single frequency).  Write a program to automatically find and eliminate it.  Remember that you’ll have to eliminate both that frequency and its inverse frequency. 

Hints: 
 - The frequency you’re looking for isn’t necessarily the one with the greatest magnitude, it’s the one that is most “out of place”.
 - Don’t just zero the frequency – having that frequency missing can be just as bad as having too much of it.  Try to estimate a reasonable magnitude using similar frequencies.
 
Make sure that you do this automatically… your program should work for any single-frequency interference in this fashion, but at a different frequency and with a different magnitude and/or phase. 

In [ ]:
# Helper Functions

# Run an edge detector on frequency domain
def anomalyDetect(image):
    
    (rows,cols) = image.shape
    
    #Create the kernel for x and y direction
    ker = np.zeros((3,3))
    ker[0,0] = -1
    ker[1,0] = -1
    ker[2,0] = -1
    ker[0,1] = -1
    ker[1,1] = 1
    ker[2,1] = -1
    ker[0,2] = -1
    ker[1,2] = -1
    ker[2,2] = -1
    
    result = np.zeros((rows,cols))
    
    # Go through image
    for i in range(1,rows-1): 
        for j in range(1,cols-1):
            mask = image[i-1:i+2,j-1:j+2]
            mask = np.multiply(ker,mask)
            value = np.sum(mask)
            result[i,j] = value
            
    return result

def freqRemove(f):
    
    F = np.fft.fft2(f)
    mags = np.absolute(F)
    
    anom = anomalyDetect(mags)
    N=len(anom) - 1
    
    result = F
    for i in range(1,N):
        for j in range(1,N):
            if anom[i][j] > 0:
                result[i][j] = average(F,i,j)
                
    r = np.fft.ifft2(result)
    r = np.absolute(r)
    
    return r
    
def average(F,i,j):
    return (F[i+1][j+1] + F[i-1][i-1] + F[i+1][i-1] + F[i-1][i+1])/4.0


In [ ]:
from scipy.ndimage import imread

filename = "interfere.png"
im = imread(filename)
im = im[:,:,0]

plt.imshow(im, cmap='gray', interpolation='none');plt.show()

brain = freqRemove(im)

plt.imshow(brain, cmap='gray', interpolation='none');plt.show()